In [2]:
import os
import re
import pdb
import torch
import logging
import numpy as np

import torch.nn.functional as F

from tqdm import tqdm, trange
from glob import glob
from mylogger import get_mylogger
from image_specific_attack import create_model

In [5]:
def load_image(image_path: str):
    filename = glob(image_path+"*.pth")
    adv_examples = []
    labels = []
    for file in tqdm(filename[0:1000]):
        adv_example = torch.load(file)
        adv_examples.append(adv_example)

        label = int(re.split('_|\.', file)[-2])
        labels.append(label)

    return adv_examples, labels
    

In [6]:
# log_path = "./log/detect"
# os.makedirs(log_path, exist_ok=True)
# logger = get_mylogger(log_path)

patch_size = 7
model = create_model(
    'ResNet18', './result/models/ResNet18_{}/model_best.pth.tar'.format(patch_size)).eval()

adv_example_path = 'result/attack/image_specific/ResNet18/ResNet18_{}/patched/'.format(
    patch_size)
adv_examples, labels = load_image(adv_example_path)

=> creating model 
100%|██████████| 1000/1000 [00:08<00:00, 122.64it/s]


In [7]:
def compute_normal_acc(model, adv_examples, labels):
    normal_acc = 0
    for i in trange(len(adv_examples)):
        adv_example = adv_examples[i]
        label = labels[i]

        adv_out = F.log_softmax(model(adv_example), dim=1)
        adv_out_probs, adv_out_labels = adv_out.max(1)
        # have already set the target label to 5.
        if adv_out_labels == label:
            normal_acc += 1
    print("\nnormal acc: ", normal_acc/len(adv_examples))

def compute_attack_acc(model, adv_examples, labels):
    attack_acc = 0
    for i in trange(len(adv_examples)):
        adv_example = adv_examples[i]
        label = labels[i]

        adv_out = F.log_softmax(model(adv_example), dim=1)
        adv_out_probs, adv_out_labels = adv_out.max(1)
        # have already set the target label to 5.
        if adv_out_labels == 5:
            attack_acc += 1
    print("\nattack acc: ", attack_acc/len(adv_examples))

In [9]:
compute_normal_acc(model, adv_examples, labels)

100%|██████████| 1000/1000 [00:02<00:00, 353.40it/s]normal acc:  0.483



In [14]:
output = model.conv1(adv_examples[0])


In [15]:
output.shape

torch.Size([1, 64, 32, 32])

In [19]:
output = output.sum(axis = 1)

In [20]:
output

tensor([[[ 0.8805,  0.0748,  0.0301,  ...,  0.7565,  0.8731, -1.4665],
         [ 0.5255,  3.8484,  4.9588,  ...,  3.6319,  3.7243,  1.6387],
         [ 1.4340,  3.3534,  3.5075,  ...,  3.4980,  3.7199,  1.0239],
         ...,
         [ 2.6459,  2.1276,  2.6825,  ...,  2.1246,  2.2442,  0.8276],
         [ 2.8909,  2.5445,  3.1157,  ...,  2.3924,  2.5949,  1.4347],
         [ 2.1683,  2.1464,  2.2724,  ...,  1.2385,  0.8842,  1.5611]]],
       device='cuda:0', grad_fn=<SumBackward1>)

In [25]:
tmp_output = output.reshape(-1)


In [26]:
topk = 115
topk_value = tmp_output[torch.topk(tmp_output,topk)[1][-1]]

In [31]:
tmp_output = torch.relu(torch.sign(output-topk_value))
tmp_output.sum()

tensor(114., device='cuda:0', grad_fn=<SumBackward0>)

In [34]:
all_one_conv = torch.nn.Conv2d(1,1,7)

In [53]:
all_one_conv.weight.data = torch.ones((1,1,7,7))

In [62]:
tmp_output.unsqueeze(0)

tensor([[[[0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          ...,
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.],
          [0., 0., 0.,  ..., 0., 0., 0.]]]], device='cuda:0',
       grad_fn=<UnsqueezeBackward0>)

In [64]:
all_one_conv(tmp_output.unsqueeze(0).to(torch.device('cpu'))).shape

torch.Size([1, 1, 26, 26])